In [18]:
import sqlite3
import pandas as pd
from datetime import datetime

In [26]:
# Define the function to parse and format dates
def parse_date_mdy_format(date_str):
    parts = date_str.split('/')
    month, day, year_component = parts
    month = int(month)
    day = int(day)
    year_component = int(year_component)
    if year_component < 28:
        year_component += 2000
    else:
        year_component += 1900
    return datetime(year=year_component, month=month, day=day).strftime('%Y-%m-%d')


# Connect to the SQLite database
conn = sqlite3.connect('../laliga.sqlite')

# Query to select the relevant columns from the matches table
query = '''
SELECT score, home_team, away_team, season, division, matchday
FROM Matches
'''

# Run the query and store the results in a DataFrame
matches_df = pd.read_sql_query(query, conn)

# Drop rows with null scores
matches_df.dropna(subset=['score'], inplace=True)

# Apply the date formatting function to the date column
#matches_df['date'] = matches_df['date'].apply(parse_date_mdy_format)

# Close the database connection
conn.close()

matches_df.head()  # Display the first few rows of the DataFrame to verify

,score,home_team,away_team,season,division,matchday
0,2:3,Arenas Club,Athletic Madrid,1928-1929,1,1
1,3:2,Espanyol,Real Unión,1928-1929,1,1
2,5:0,Real Madrid,Catalunya,1928-1929,1,1
3,1:1,Donostia,Athletic,1928-1929,1,1
4,0:2,Racing,Barcelona,1928-1929,1,1


In [35]:
# Helper function to determine match outcomes 
def determine_outcome(home_goals, away_goals):
    if home_goals > away_goals:
        return 'W', 'L'
    elif home_goals < away_goals:
        return 'L', 'W'
    else:
        return 'T', 'T'

In [37]:
def calculate_single_season_division_standings(season, division, matches_df):
    # Filter matches for the given season and division
    season_matches = matches_df[(matches_df['season'] == season) & (matches_df['division'] == division)].sort_values('matchday')
    matchdays = season_matches['matchday'].unique()

    # Initialize the standings dict with teams
    teams = pd.unique(season_matches[['home_team', 'away_team']].values.ravel('K'))
    standings_tracker = {team: {'GF': 0, 'GA': 0, 'GD': 0, 'W': 0, 'L': 0, 'T': 0, 'PTS': 0, 'last_5': []} for team in teams}

    # List to collect matchday standings
    all_standings = []

    # Process each matchday
    for matchday in matchdays:
        matchday_matches = season_matches[season_matches['matchday'] == matchday]
        for index, match in matchday_matches.iterrows():
            home_team, away_team = match['home_team'], match['away_team']
            home_goals, away_goals = map(int, match['score'].split(':'))
            home_outcome, away_outcome = determine_outcome(home_goals, away_goals)

            # Update goals for and against
            standings_tracker[home_team]['GF'] += home_goals
            standings_tracker[away_team]['GF'] += away_goals
            standings_tracker[home_team]['GA'] += away_goals
            standings_tracker[away_team]['GA'] += home_goals

            # Update last 5 matches
            if matchday > 1: # Only update if it's not the first matchday
                standings_tracker[home_team]['last_5'].insert(0, home_outcome)
                standings_tracker[away_team]['last_5'].insert(0, away_outcome)

            # Ensure last_5 lists do not exceed 5 matches
            standings_tracker[home_team]['last_5'] = standings_tracker[home_team]['last_5'][:5]
            standings_tracker[away_team]['last_5'] = standings_tracker[away_team]['last_5'][:5]

            # Update wins, losses, ties, and points
            if home_goals > away_goals:  # Home win
                standings_tracker[home_team]['W'] += 1
                standings_tracker[home_team]['PTS'] += 3
                standings_tracker[away_team]['L'] += 1
            elif home_goals < away_goals:  # Away win
                standings_tracker[away_team]['W'] += 1
                standings_tracker[away_team]['PTS'] += 3
                standings_tracker[home_team]['L'] += 1
            else:  # Tie
                standings_tracker[home_team]['T'] += 1
                standings_tracker[home_team]['PTS'] += 1
                standings_tracker[away_team]['T'] += 1
                standings_tracker[away_team]['PTS'] += 1

        # Calculate goal difference
        for team in teams:
            standings_tracker[team]['GD'] = standings_tracker[team]['GF'] - standings_tracker[team]['GA']

        # Create standings DataFrame for the current matchday
        matchday_standings = (pd.DataFrame.from_dict(standings_tracker, orient='index')
                                .reset_index()
                                .rename(columns={'index': 'Team'}))
        matchday_standings['Matchday'] = matchday
        matchday_standings['Season'] = season
        matchday_standings['Division'] = division

        # Sort standings
        matchday_standings.sort_values(by=['PTS', 'GD', 'GF'], ascending=[False, False, False], inplace=True)
        matchday_standings['Rank'] = matchday_standings.reset_index().index + 1

        # Append to the list
        all_standings.append(matchday_standings)

    # Concatenate all matchday standings
    final_standings = pd.concat(all_standings, ignore_index=True)
    # Reorder columns
    final_standings = final_standings[['Season', 'Division', 'Matchday', 'Rank', 'Team', 'GF', 'GA', 'GD', 'W', 'L', 'T', 'PTS', 'last_5']]
    
    return final_standings

#calculate_single_season_division_standings('2020-2021', 1, matches_df).head(60)

In [39]:
def calculate_all_seasons_divisions_standings_optimized(matches_df):
    # Initialize the final DataFrame
    final_all_standings = pd.DataFrame()
    
    # Process each season and division without explicit loops
    for (season, division), group_df in matches_df.groupby(['season', 'division']):
        season_division_standings = calculate_single_season_division_standings(season, division, group_df)
        final_all_standings = pd.concat([final_all_standings, season_division_standings], ignore_index=True)
    
    return final_all_standings

all_seasons_divisions_standings_optimized = calculate_all_seasons_divisions_standings_optimized(matches_df)
all_seasons_divisions_standings_optimized

,Season,Division,Matchday,Rank,Team,GF,GA,GD,W,L,T,PTS,last_5
0,1928-1929,1,1,1,Real Madrid,5,0,5,1,0,0,3,[W]
1,1928-1929,1,1,2,Barcelona,2,0,2,1,0,0,3,[L]
2,1928-1929,1,1,3,Espanyol,3,2,1,1,0,0,3,[L]
3,1928-1929,1,1,4,Athletic Madrid,3,2,1,1,0,0,3,[L]
4,1928-1929,1,1,5,Donostia,1,1,0,0,0,1,1,[W]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
96213,2021-2022,2,3,18,CD Leganés,1,3,-2,0,2,1,1,"[L, T]"
96214,2021-2022,2,3,19,Real Zaragoza,0,2,-2,0,1,1,1,[L]
96215,2021-2022,2,3,20,Burgos CF,0,2,-2,0,2,1,1,"[L, T]"
96216,2021-2022,2,3,21,FC Cartagena,1,5,-4,0,2,0,0,[L]


In [46]:
def calculate_single_season_division_standings(season, division, matches_df):
    # Filter matches for the given season and division
    season_matches = matches_df[(matches_df['season'] == season) & (matches_df['division'] == division)].sort_values('matchday')
    matchdays = season_matches['matchday'].unique()

    # Initialize the standings dict with teams
    teams = pd.unique(season_matches[['home_team', 'away_team']].values.ravel('K'))
    standings_tracker = {team: {'GF_home': 0, 'GA_home': 0, 'GF_away': 0, 'GA_away': 0, 'W': 0, 'L': 0, 'T': 0, 'PTS': 0, 'last_5': []} for team in teams}

    # Function to determine match outcome
    def determine_outcome(home_goals, away_goals):
        if home_goals > away_goals:
            return 'W', 'L'
        elif home_goals < away_goals:
            return 'L', 'W'
        return 'T', 'T'

    # Function to calculate relative strength
    def calculate_relative_strength(standings_df, matchday):
        # Calculate the maximum possible points so far for each team
        max_points_so_far = matchday * 3
        # Calculate the relative strength for each team as the percentage of points obtained out of the total possible
        standings_df['Relative_Strength'] = standings_df['PTS'] / max_points_so_far * 100
        return standings_df

    # List to collect matchday standings
    all_standings = []

    # Process each matchday
    for matchday in matchdays:
        matchday_matches = season_matches[season_matches['matchday'] == matchday]
        for index, match in matchday_matches.iterrows():
            home_team, away_team = match['home_team'], match['away_team']
            home_goals, away_goals = map(int, match['score'].split(':'))
            home_outcome, away_outcome = determine_outcome(home_goals, away_goals)

            # Update goals for and against at home and away
            standings_tracker[home_team]['GF_home'] += home_goals
            standings_tracker[away_team]['GF_away'] += away_goals
            standings_tracker[home_team]['GA_home'] += away_goals
            standings_tracker[away_team]['GA_away'] += home_goals

            # Update last 5 matches
            if matchday > 1:  # Only update if it's not the first matchday
                standings_tracker[home_team]['last_5'].insert(0, home_outcome)
                standings_tracker[away_team]['last_5'].insert(0, away_outcome)

            # Ensure last_5 lists do not exceed 5 matches
            standings_tracker[home_team]['last_5'] = standings_tracker[home_team]['last_5'][:5]
            standings_tracker[away_team]['last_5'] = standings_tracker[away_team]['last_5'][:5]

            # Update wins, losses, ties, and points
            if home_goals > away_goals:  # Home win
                standings_tracker[home_team]['W'] += 1
                standings_tracker[home_team]['PTS'] += 3
                standings_tracker[away_team]['L'] += 1
            elif home_goals < away_goals:  # Away win
                standings_tracker[away_team]['W'] += 1
                standings_tracker[away_team]['PTS'] += 3
                standings_tracker[home_team]['L'] += 1
            else:  # Tie
                standings_tracker[home_team]['T'] += 1
                standings_tracker[home_team]['PTS'] += 1
                standings_tracker[away_team]['T'] += 1
                standings_tracker[away_team]['PTS'] += 1

        # Calculate goal difference for each team
        for team in teams:
            standings_tracker[team]['GD'] = standings_tracker[team]['GF_home'] + standings_tracker[team]['GF_away'] - \
                                             standings_tracker[team]['GA_home'] - standings_tracker[team]['GA_away']

        # Create standings DataFrame for the current matchday
        matchday_standings = (pd.DataFrame.from_dict(standings_tracker, orient='index')
                                .reset_index()
                                .rename(columns={'index': 'Team'}))
        matchday_standings['Matchday'] = matchday
        matchday_standings['Season'] = season
        matchday_standings['Division'] = division

        # Sort standings
        matchday_standings.sort_values(by=['PTS', 'GD', 'GF_home', 'GF_away'], ascending=[False, False, False, False], inplace=True)
        matchday_standings['Rank'] = matchday_standings.reset_index(drop=True).index + 1

        # Calculate the relative strength
        matchday_standings = calculate_relative_strength(matchday_standings, matchday)

        # Append to the list
        all_standings.append(matchday_standings)

    # Concatenate all matchday standings
    final_standings = pd.concat(all_standings, ignore_index=True)
    # Reorder columns
    final_standings = final_standings[['Season', 'Division', 'Matchday', 'Rank', 'Team', 'GD', 'GF_home', 'GA_home', 'GF_away', 'GA_away', 'W', 'L', 'T', 'PTS', 'last_5', 'Relative_Strength']]
    
    return final_standings

# Use this function to generate the standings for a given season and division
#calculate_single_season_division_standings('2020-2021', 1, matches_df)

def calculate_all_seasons_divisions_standings(matches_df):
    # Initialize the final DataFrame
    final_all_standings = pd.DataFrame()

    # Process each season and division without explicit loops
    for (season, division), group_df in matches_df.groupby(['season', 'division']):
        season_division_standings = calculate_single_season_division_standings(season, division, group_df)
        final_all_standings = pd.concat([final_all_standings, season_division_standings], ignore_index=True)

    return final_all_standings

features_df =calculate_all_seasons_divisions_standings(matches_df)

In [44]:
def encode_match_outcome(score):
    home_goals, away_goals = map(int, score.split(':'))
    if home_goals > away_goals:
        return '1'  # Home win
    elif home_goals < away_goals:
        return '2'  # Away win
    else:
        return 'X'  # Draw

# Apply the encoding function to the score column
matches_df['outcome'] = matches_df['score'].apply(encode_match_outcome)


,score,home_team,away_team,season,division,matchday,home_goals,away_goals,outcome
0,2:3,Arenas Club,Athletic Madrid,1928-1929,1,1,2,3,2
1,3:2,Espanyol,Real Unión,1928-1929,1,1,3,2,1
2,5:0,Real Madrid,Catalunya,1928-1929,1,1,5,0,1
3,1:1,Donostia,Athletic,1928-1929,1,1,1,1,X
4,0:2,Racing,Barcelona,1928-1929,1,1,0,2,2
...,...,...,...,...,...,...,...,...,...
48345,2:1,Ponferradina,Girona,2021-2022,2,3,2,1,1
48346,2:1,SD Amorebieta,UD Almería,2021-2022,2,3,2,1,1
48347,0:2,CD Lugo,Real Valladolid,2021-2022,2,3,0,2,2
48348,0:0,Real Sociedad B,CF Fuenlabrada,2021-2022,2,3,0,0,X


In [61]:
# Perform the first merge for Home Team
matches_df = pd.merge(
    left=matches_df,
    right=features_df,
    how='left',
    left_on=['season', 'division', 'matchday', 'home_team'],
    right_on=['Season', 'Division', 'Matchday', 'Team'],
    suffixes=('', '_home')  # Add a suffix only to the columns from the right DataFrame
)

# Drop the duplicated columns from the standings dataframe
columns_to_drop = ['Season', 'Division', 'Matchday', 'Team', 'Rank', 'last_5']
matches_df.drop(columns=columns_to_drop, inplace=True)

# Rename the columns from the first merge to indicate they are for the home team
home_feature_columns = {
    'Relative_Strength': 'Home_Relative_Strength',
    'GD': 'Home_GD',
    'GF_home': 'Home_GF',
    'GA_home': 'Home_GA',
    'Rank_home': 'Home_Rank',
    'last_5': 'Home_last_5',
    # Include other feature columns as needed
}
matches_df.rename(columns=home_feature_columns, inplace=True)

# Perform the second merge for Away Team
matches_df = pd.merge(
    left=matches_df,
    right=features_df,
    how='left',
    left_on=['season', 'division', 'matchday', 'away_team'],
    right_on=['Season', 'Division', 'Matchday', 'Team'],
    suffixes=('', '_away')  # Add a suffix only to the columns from the right DataFrame
)

# Drop the duplicated columns again
matches_df.drop(columns=columns_to_drop, inplace=True)

# Rename the columns from the second merge to indicate they are for the away team
away_feature_columns = {
    'Relative_Strength_away': 'Away_Relative_Strength',
    'GD_away': 'Away_GD',
    'GF_away_away': 'Away_GF',
    'GA_away_away': 'Away_GA',
    'Rank_away': 'Away_Rank',
    'last_5': 'Away_last_5',
    # Include other feature columns as needed
}
matches_df.rename(columns=away_feature_columns, inplace=True)

matches_df

,score,home_team,away_team,season,division,matchday,home_goals,away_goals,outcome,Season_x,...,GD,GF_home,GA_home,Away_GF,Away_GA,W_away,L_away,T_away,PTS_away,Relative_Strength
0,2:3,Arenas Club,Athletic Madrid,1928-1929,1,1,2,3,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3:2,Espanyol,Real Unión,1928-1929,1,1,3,2,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5:0,Real Madrid,Catalunya,1928-1929,1,1,5,0,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1:1,Donostia,Athletic,1928-1929,1,1,1,1,X,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0:2,Racing,Barcelona,1928-1929,1,1,0,2,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47995,2:1,Ponferradina,Girona,2021-2022,2,3,2,1,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47996,2:1,SD Amorebieta,UD Almería,2021-2022,2,3,2,1,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47997,0:2,CD Lugo,Real Valladolid,2021-2022,2,3,0,2,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47998,0:0,Real Sociedad B,CF Fuenlabrada,2021-2022,2,3,0,0,X,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
